# Create CNN Model and Optimizie using Keras Tuner

In [ ]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
# Scaling our images between 0 to 1
train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images), 28 ,28, 1)
test_images = test_images.reshape(len(test_images), 28 ,28, 1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

In [12]:
tuner_search = RandomSearch(build_model, objective ='val_accuracy', max_trials=5, directory='output',project_name='Mnist Fashion')

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split =0.1)

Trial 5 Complete [00h 27m 06s]
val_accuracy: 0.909333348274231

Best val_accuracy So Far: 0.909333348274231
Total elapsed time: 01h 23m 26s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 96)        2496      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        153664    
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 96)                2457696   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2,614,826
Trainable params: 2,614,826
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 543s 322ms/step - loss: 0.1715 - accuracy: 0.9360 - val_loss: 0.2725 - val_accuracy: 0.9065
Epoch 5/10
1688/1688 [==============================] - 545s 323ms/step - loss: 0.1343 - accuracy: 0.9492 - val_loss: 0.2605 - val_accuracy: 0.9095
Epoch 6/10
1688/1688 [==============================] - 541s 321ms/step - loss: 0.1057 - accuracy: 0.9603 - val_loss: 0.3032 - val_accuracy: 0.9152
Epoch 7/10
1688/1688 [==============================] - 541s 321ms/step - loss: 0.0804 - accuracy: 0.9695 - val_loss: 0.3838 - val_accuracy: 0.9085
Epoch 8/10
1688/1688 [==============================] - 542s 321ms/step - loss: 0.0606 - accuracy: 0.9767 - val_loss: 0.3746 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 541s 321ms/step - loss: 0.0481 - accuracy: 0.9820 - val_loss: 0.4312 - val_accuracy: 0.9062
Epoch 10/10
1688/1688 [==============================] - 545s 323ms/step - loss: 0.0428 - accuracy: 0.9846 - val